In [1]:
import sys

import os

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Add the project root directory to the Python path
sys.path.append(project_root)

In [2]:
import pandas as pd


project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Define the path to the data directory
data_dir = os.path.join(project_root, 'data')

# Example: Access a specific data file in the data directory
data_file_path = os.path.join(data_dir, 'RADCURE_challenge_clinical.csv')
df = pd.read_csv(data_file_path)

In [3]:
from jarvais.analyzer import Analyzer
from pprint import pprint

df.drop(columns=["Study ID", "split"], inplace=True)
df.rename(columns={'survival_time': 'time', 'death':'event'}, inplace=True)

config = Analyzer.dry_run(df)

pprint(config)

/home/joshua-siraj/Documents/CDI/jarvais/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config file not found. Creating custom...
Used a heuristic to define categorical and continuous columns. Please review!


Feature Types:
  - Categorical: ['N Stage', 'Disease Site', 'Stage', 'Sex', 'T Stage', 'EGFRI', 'Chemotherapy', 'HPV Combined', 'Dose', 'event']
  - Continuous: ['age at dx', 'time']


Outlier Analysis:
  - Outliers found in N Stage: ['N3a: 14 out of 2552', 'NX: 1 out of 2552']
  - Outliers found in Disease Site: ['esophagus: 22 out of 2552', 'salivary glands: 4 out of 2552']
  - Outliers found in Stage: ['IV: 6 out of 2549', 'IIIC: 2 out of 2549', 'IIIA: 2 out of 2549', 'IIA: 1 out of 2549']
  - No Outliers found in Sex
  - Outliers found in T Stage: ['T2 (2): 1 out of 2552', 'TX: 1 out of 2552', 'T3 (2): 1 out of 2552']
  - No Outliers found in EGFRI
  - No Outliers found in Chemotherapy
  - No Outliers found in HPV Combined
  - Outliers found in Dose: ['50.0: 9 out of 2552', '69.96: 2 out of 2552', '50.8: 1 out of 2552', '55.0: 1 out of 2552', '53.55: 1 out of 25

In [4]:
import yaml
from pathlib import Path

config['columns']['categorical'].remove('Dose')
config['columns']['continuous'].append('Dose') 

pprint(config)

analyzer_path = Path('radcure_outputs/analyzer')
analyzer_path.mkdir(parents=True, exist_ok=True)

with open(analyzer_path / 'config.yaml', 'w') as f:
            yaml.dump(config, f)

{'columns': {'categorical': ['N Stage',
                             'Disease Site',
                             'Stage',
                             'Sex',
                             'T Stage',
                             'EGFRI',
                             'Chemotherapy',
                             'HPV Combined',
                             'event'],
             'continuous': ['age at dx', 'time', 'Dose'],
             'date': [],
             'other': []},
 'mapping': {'Chemotherapy': {'0': '0', '1': '1'},
             'Disease Site': {'esophagus': 'Other',
                              'hypopharynx': 'hypopharynx',
                              'larynx': 'larynx',
                              'lip & oral cavity': 'lip & oral cavity',
                              'nasal cavity': 'nasal cavity',
                              'nasopharynx': 'nasopharynx',
                              'oropharynx': 'oropharynx',
                              'paranasal sinus': 'paranasal

In [5]:
from jarvais.analyzer import Analyzer

analyzer = Analyzer(df, task='time_to_event', target_variable='event', one_hot_encode=True, output_dir='./radcure_outputs/analyzer', config='radcure_outputs/analyzer/config.yaml')

analyzer.run()

Feature Types:
  - Categorical: ['N Stage', 'Disease Site', 'Stage', 'Sex', 'T Stage', 'EGFRI', 'Chemotherapy', 'HPV Combined', 'event']
  - Continuous: ['age at dx', 'time', 'Dose']


Outlier Analysis:
  - Outliers found in N Stage: ['N3a: 14 out of 2552', 'NX: 1 out of 2552']
  - Outliers found in Disease Site: ['esophagus: 22 out of 2552', 'salivary glands: 4 out of 2552']
  - Outliers found in Stage: ['IV: 6 out of 2549', 'IIIC: 2 out of 2549', 'IIIA: 2 out of 2549', 'IIA: 1 out of 2549']
  - No Outliers found in Sex
  - Outliers found in T Stage: ['T2 (2): 1 out of 2552', 'TX: 1 out of 2552', 'T3 (2): 1 out of 2552']
  - No Outliers found in EGFRI
  - No Outliers found in Chemotherapy
  - No Outliers found in HPV Combined
  - No Outliers found in event

Applying changes from config...

╒══════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                      │                   │ Missing   │ Overall     │
╞══════════════════════╪═══════════════════╪═══════════╪

In [2]:
from jarvais.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()

Possible Bias in Disease Site:



<lifelines.CoxPHFitter: fitted with 511 total observations, 349 right-censored observations>
             duration col = 'time'
                event col = 'event'
                penalizer = 0.0001
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 511
number of events observed = 162
   partial log-likelihood = -920.54
         time fit was run = 2025-01-31 17:35:56 UTC

---
                                coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                       
Disease Site_Other              1.34      3.82      5.36           -9.17           11.85                0.00            1.40e+05
Disease Site_hypopharynx        1.01      2.74      5.32           -9.42           11.43                0.00            92426.06
Disease Site_larynx             0.04      1.04      5.32          -10.38           10.46                0.00            34857.41
Disease Site_lip & oral cavity  0.61      1.85      5.33           -9.82           11.05                0.00            63035.94
Disease Site_nasal cavity       0.53      1.70      5.33           -9.91           10.98                0.00            58726.95
Disease Site_nasopharynx       -0.87      0.42      5.32          -11.31            9.56                0.00            14173.91
Disease Site_oropharynx        -0.13      0.88      5.32          -10.54           10.29                0.00            29560.41
Disease Site_paranasal sinus    1.03      2.81      5.36           -9.48           11.54                0.00            1.03e+05

                                cmp to     z    p  -log2(p)
covariate                                                  
Disease Site_Other                0.00  0.25 0.80      0.32
Disease Site_hypopharynx          0.00  0.19 0.85      0.23
Disease Site_larynx               0.00  0.01 0.99      0.01
Disease Site_lip & oral cavity    0.00  0.12 0.91      0.14
Disease Site_nasal cavity         0.00  0.10 0.92      0.12
Disease Site_nasopharynx          0.00 -0.16 0.87      0.20
Disease Site_oropharynx           0.00 -0.02 0.98      0.03
Disease Site_paranasal sinus      0.00  0.19 0.85      0.24
---
Concordance = 0.61
Partial AIC = 1857.08
log-likelihood ratio test = 32.33 on 8 df
-log2(p) of ll-ratio test = 13.58